In [2]:
from bs4 import BeautifulSoup
import requests
import re
import lxml.html as lh
from urllib.request import urlretrieve
import pandas as pd
import urllib
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import json
import textract

In [3]:
"""
f = open('All_Programs.csv','r')
final_course_list=[]
content = f.readlines()
for c in content:
    if(c[0].isupper() and c[1].isupper() and c[2].isupper()):
        final_course_list.append(c[0:8])
"""
program_df = pd.read_csv('All_Programs.csv')
program_df.columns =list(map(lambda x: x.strip(), program_df.columns))
#f.close()

In [43]:
def is_honours(program):
    return "H" in program

YEARS = range(2018,2020)
for year in YEARS:
    for program in program_df['Code']:
        #if(not is_honours(program)):
        #    continue
        if(len(program) > 5):
            continue
        print(program)
        main_page = requests.get("https://programsandcourses.anu.edu.au/"+str(year)+"/program/"+program)
        soup = BeautifulSoup(main_page.content, 'html.parser')
        course_dict=dict()
        course_dict["year"] = year
        required = dict()
        required["x_from_here"] = [] 
        required["x_from_category"] = []
        required["one_from_m/m/s"] = []
        for title in soup.find_all("span",class_ = "intro__degree-title__component"):
            course_dict["name"] = title.get_text()
        for units in soup.find_all("li",class_ = "degree-summary__requirements-units"):
            course_dict["units"] = [int(s) for s in units.get_text().split() if s.isdigit()][0]
        
        if(is_honours(program)):
            curr_node = ""
            curr_num = None
            for s in soup.find_all("p"):
                
                st = str(s)
                if("units from completion" in st) and not ('elective' in st):
                        curr_node = st[:-3].split(">")[1]
                        curr_num =st.split(" ")[0]
                        print(curr_node)
                        
                if("margin-left: 80px;" in st):
                    print(st)
                    for statement in str(s).split(">"):
                        if(curr_node == ""):
                            break
                        if(curr_node == "mms")and ("href" in statement):
                            required["one_from_m/m/s"].append(statement[-9:])
                            continue
                        if(curr_node != "") and ("href" in statement):
                            required["x_from_here"][-1]["courses"].append(statement[-9:])
                            if(curr_num == '6'):
                                break
                        if("following" in statement) and (("major" in statement) or ("minor" in statement) or ("specialisation" in statement)):
                            curr_node = "mms"
                            continue
                        if("units from completion" in statement) and not ('elective' in statement):
                            curr_node = statement[:-3]
                            curr_num =statement.split(" ")[0]
                            if("List 1" in statement):
                                required["x_from_category"].append({"num":curr_num,"code":['List 1']})

                            required["x_from_here"].append({"num":curr_num,"courses":[]})
        else:
            for s in soup.find_all("div",{"id": "study"}):
                curr_node = ""
                curr_num = None
                for statement in str(s).split(">"):
                    if(curr_node == "mms")and ("href" in statement):
                        required["one_from_m/m/s"].append(statement[-9:])
                        continue
                    if(curr_node != "") and ("href" in statement):
                        required["x_from_here"][-1]["courses"].append(statement[-9:])
                        if(curr_num == '6'):
                            break
                    if("following" in statement) and (("major" in statement) or ("minor" in statement) or ("specialisation" in statement)):
                        curr_node = "mms"
                        continue
                    if("units from completion" in statement) and not ('elective' in statement):
                        curr_node = statement[:-3]
                        curr_num =statement.split(" ")[0]
                        if("List 1" in statement):
                            required["x_from_category"].append({"num":curr_num,"code":['List 1']})

                        required["x_from_here"].append({"num":curr_num,"courses":[]})



            course_dict["required"] = required
            print(course_dict)
            #s.split(">")
            #bs = BeautifulSoup(s.content,'html.parser')
            #for p in bs.find_all("p"):
            #    print(p)
        
# look for a regex which is like > (some number) units from completion of a course from the following list.
# Then get all the HRefs in the proceeding list.
# format in xml
# readd for all degrees


BACCT
{'year': 2018, 'required': {'x_from_category': [{'num': '6', 'code': ['List 1']}], 'x_from_here': [{'num': '24', 'courses': []}, {'num': '30', 'courses': ['BUSN1002"', 'FINM1001"', 'BUSN2011"', 'BUSN2015"', 'BUSN3001"']}, {'num': '18', 'courses': ['BUSN1101"', 'BUSN2101"', 'BUSN2036"', 'BUSN3008"', 'BUSN3017"', 'INFS2005"']}, {'num': '18', 'courses': ['BUSN3002"', 'BUSN3003"', 'BUSN3006"', 'BUSN3008"', 'BUSN3014"', 'BUSN3017"', 'BUSN3051"']}, {'num': '6', 'courses': ['BUSN1101"']}], 'one_from_m/m/s': []}, 'units': 144, 'name': 'Bachelor of Accounting'}
HACCT


KeyboardInterrupt: 

In [ ]:
course_codes = list(pd.read_csv("courses.csv")['\n                    Code\n                '])
course_names = list(pd.read_csv("courses.csv")['\n                    Title\n                '])
course_descriptions=[]
course_codes=final_course_list
for i in range(0,len(final_course_list)):
    if(course_codes[i] in seen):
        break
    seen.add(i)
    course=course_codes[i]
    print("Downloading reports for:",course)
    main_page = requests.get("https://programsandcourses.anu.edu.au/2018/course/"+course)
    soup = BeautifulSoup(main_page.content, 'html.parser')
    desc=""
    req=""
    outcomes=""
    req_description=""
    full_text=soup.get_text()
    for s in soup.find_all("div","introduction"):
        desc = s.get_text()
    for s in soup.find_all("div","requisite"):
        req=parse_prereq(s.get_text())
        
        req_description=s.get_text()
        
    for s in soup.find_all("ol"):
        outcomes = s.get_text()    

   
    
    course_descriptions.append([course,course_names[i],desc,outcomes,req,req_description,full_text])
